   Copyright 2015-2023, University of Bern, Laboratory for High Energy Physics and Theodor Kocher Institute, M. Vladymyrov

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.


# 0. libs & utils

In [ ]:
import skimage
from skimage.filters import gaussian
from scipy.signal import argrelmin
import numpy as np
import matplotlib.pyplot as plt
import utils.imgio as iio
import utils.imgio

from utils.CZI_image import CZI_image

import os
import glob

import shutil
import pickle

In [ ]:
from aicsimageio import AICSImage
import re
from PIL import Image
from utils import imgio as iio

In [ ]:
def save_pckl(d, fname):
    with open(fname, 'wb') as f:
        pickle.dump(d, f, protocol=pickle.DEFAULT_PROTOCOL)
def load_pckl(fname):
    with open(fname, 'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
def ish(im, vmin=0, vmax=255):
    plt.imshow(im, cmap='gray', vmin=vmin, vmax=vmax)
    plt.show()
    plt.close()

In [ ]:
def read_mp_tiff(path):
    """
    Args:
        path (str) : path to the images, e.g. `/path/to/stacks/img.png`

    Returns:
        image (np.ndarray): image, DHWC
    """
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img))
    return np.array(images)

In [ ]:
def minmax(a):
    return np.min(a), np.max(a)

# 1. Interface

In [ ]:
def save_as_8bit_tifs(root_dir, ds_name, ci):
    # format : <ds_name>_t0xcxmx.tif
    
    n_c = ci.n_c
    n_tile = ci.n_tile
    n_t = ci.n_t
    
    
    tmpl_t = 't%0' + '%d' % len('%d' % n_t) + 'd'
    tmpl_ch = 'c%0' + '%d' % len('%d' % n_c) + 'd'
    tmpl_tl = 'm%0' + '%d' % len('%d' % n_tile) + 'd'
    
    path = os.path.join(root_dir, ds_name)
    os.makedirs(path, exist_ok=True)
    
    for ch in range(n_c):
        arr = [minmax(ci.get_frame(tile=tile, t=t, c=ch)) for t in range(n_t) for tile in range(n_tile)]
        arr = np.array(arr).T
        v_min = np.min(arr[0])
        v_max = np.max(arr[1])
        del arr
        
        print(ch, v_min, v_max)
        
        for t in range(n_t):
            for tile in range(n_tile):
        
                sfx = '_'
                sfx += tmpl_t % (t+1)

                sfx += tmpl_ch % (ch+1)

                sfx += tmpl_tl % (tile+1)

                name = os.path.join(path, ds_name + sfx + '.tif')
                
                im = ci.get_frame(tile=tile, t=t, c=ch)
                im_normed = (im-v_min) / (v_max - v_min)
                im_8b = (255*im_normed.clip(0, 1)).astype(np.uint8)
                
                iio.save_image(im_8b, name)

In [ ]:
def dataset_from_czi(root_dir, image_file_name):
    """
    Generates images from 2 merged tiles in x direction
    """
    # read
    print('reading')
    ci = CZI_image(image_file_name)
    ds_name = os.path.splitext(os.path.basename(fname))[0]
    save_as_8bit_tifs(root_dir, ds_name, ci)
    del ci

## tests

In [ ]:
%%time
#test:

fname = r'g:\IVFCA\Dixy\FC8_Untreated.czi'
root_dir = os.path.dirname(fname)

ci = CZI_image(file_name=fname)

In [ ]:
root_dir = os.path.dirname(fname)

In [ ]:
ds_name = os.path.splitext(os.path.basename(fname))[0]

In [ ]:
save_as_8bit_tifs(root_dir, ds_name, ci)

# 2. batch proc

In [ ]:
src_dir = r'D:\data\set\path'
root_dir = src_dir

In [ ]:
all_ds = list(glob.glob(src_dir+ '\\**\\'+'*.czi', recursive=True))
for name in all_ds:
    print(name)

In [ ]:
%%time

for fname in all_ds:
    dataset_from_czi(root_dir, fname)